In [3]:
#import minsearch
import json

In [4]:
with open('../01-intro/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

### Index Documents - Archived

In [6]:
# index = minsearch.Index(
#         text_fields=["question", "text", "section"],  #use for performant search, look for text that can help answer the question
#         keyword_fields=["course"]  #which documents to look in
#         )

# index.fit(documents)

# boost = {"question": 3.0, "section": 0.5} # if a section of document has relevant terms, attirbute more importance to those sections. For example, question field is 3x more important than text field

# results = index.search(
#     query = q,
#     boost_dict=boost,
#     num_results = 5
#     )

# results_de_zoom = index.search(
#     query=q,
#     filter_dict={'course': 'data-engineering-zoomcamp'},
#     boost_dict=boost,
#     num_results = 5
# )

# results_de_zoom

### Use GPT-4o model for generic response

In [7]:
from openai import OpenAI

from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('API_KEY')

In [8]:
openai_client = OpenAI()

### Use GPT-4o model for specific response

### Modularize the code

In [9]:
def build_prompt(query, search_results):
    
    context = ""
    for doc in search_results: 
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    
    prompt = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. Use only the facts from the CONTEXT when answering the question.

QUESTION: {query} 

CONTEXT: {context}

""".strip()
    
    return prompt

In [10]:
def llm(prompt):
    response = openai_client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role": "user", "content": prompt}]
)

    return response.choices[0].message.content 

In [22]:
query = "how do i run kafka?"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query=query, search_results=search_results )
    return llm(prompt=prompt)

### Using Vectorsearch

In [11]:
from qdrant_client import QdrantClient, models

/workspaces/llms/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
qdrant_client = QdrantClient("http://localhost:6333")

In [14]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"


In [15]:
collection_name = "zoomcamp-faq"

qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [16]:
points = []
id = 0

for idx, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id = idx,
        vector = vector, # embed text locally with jinaai from FastEmbed
        payload = doc


    )

    points.append(point)

In [18]:
qdrant_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files: 100%|██████████| 5/5 [00:01<00:00,  3.63it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [19]:
qdrant_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword" # exact matching on string metadata fields

)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [29]:
def vector_search_in_course(query, course="mlops-zoomcamp", limit=1):
    print('vector search is used')
    results = qdrant_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter(
            must = [
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]

        ),
        limit=limit,
        with_payload=True

    )

    payload_results = []

    for point in results.points:
        payload_results.append(point.payload)

    return payload_results

In [27]:
def rag_vector(query, course = 'data-engineering-zoomcamp'):
    search_results = vector_search_in_course(query, course)
    prompt = build_prompt(query=query, search_results=search_results )
    return llm(prompt=prompt)

In [30]:
query = 'how do i run kafka?'

rag_vector(query=query)

vector search is used


'To run a Kafka producer in the terminal, navigate to the project directory and execute the following command:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```'